In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os



In [2]:
PROJECT_ROOT = Path(r"C:\Users\joshu\OneDrive\Desktop\Projects and Analysis 2\crime-housing-affordability")
os.chdir(PROJECT_ROOT)


In [3]:
from config.financial_assumptions import (
    MONTHLY_GROSS_INCOME,
    FEDERAL_EFFECTIVE_TAX,
    PAYROLL_TAX,
    NON_HOUSING_MONTHLY,
    MONTHLY_SAVINGS_TARGET,
    HOUSING_COMFORTABLE_RATIO,
    HOUSING_STRETCHED_RATIO,
    MORTGAGE_RATE,
    MORTGAGE_TERM_YEARS,
    DOWN_PAYMENT_RATE,
    PROPERTY_TAX_RATE,
    HOME_INSURANCE_MONTHLY
)


In [4]:
zhvi_raw = pd.read_csv("data/raw/zillow/zillow_zhvi_sfr_midtier_metro.csv")
zori_raw = pd.read_csv("data/raw/zillow/zillow_zori_rent_metro.csv")

print("ZHVI raw shape:", zhvi_raw.shape)
print("ZORI raw shape:", zori_raw.shape)



ZHVI raw shape: (895, 316)
ZORI raw shape: (556, 136)


In [5]:
id_cols = ["RegionID", "SizeRank", "RegionName", "RegionType", "StateName"]

zhvi_long = zhvi_raw.melt(id_vars=id_cols, var_name="date", value_name="home_value")
zori_long = zori_raw.melt(id_vars=id_cols, var_name="date", value_name="rent")

zhvi_long["date"] = pd.to_datetime(zhvi_long["date"])
zori_long["date"] = pd.to_datetime(zori_long["date"])

print("ZHVI long:", zhvi_long.shape)
print("ZORI long:", zori_long.shape)


ZHVI long: (278345, 7)
ZORI long: (72836, 7)


In [6]:
latest_zhvi = zhvi_long["date"].max()
latest_zori = zori_long["date"].max()
latest_date = min(latest_zhvi, latest_zori)

print("Latest ZHVI:", latest_zhvi)
print("Latest ZORI:", latest_zori)
print("Using latest_date:", latest_date)


Latest ZHVI: 2025-11-30 00:00:00
Latest ZORI: 2025-11-30 00:00:00
Using latest_date: 2025-11-30 00:00:00


In [7]:
zhvi_latest = zhvi_long[zhvi_long["date"] == latest_date].copy()
zori_latest = zori_long[zori_long["date"] == latest_date].copy()

zhvi_latest.to_csv("data/processed/zhvi_latest.csv", index=False)
zori_latest.to_csv("data/processed/zori_latest.csv", index=False)

print("Saved processed Zillow latest files.")
print("ZHVI latest shape:", zhvi_latest.shape)
print("ZORI latest shape:", zori_latest.shape)


Saved processed Zillow latest files.
ZHVI latest shape: (895, 7)
ZORI latest shape: (556, 7)


In [8]:
population = pd.read_csv("data/processed/population_2024_metro.csv")
print("Population shape:", population.shape)
population.head()


Population shape: (393, 2)


,metro_name,population_2024
0,".Abilene, TX",184278.0
1,".Akron, OH",702209.0
2,".Albany, GA",145451.0
3,".Albany, OR",132474.0
4,".Albany-Schenectady-Troy, NY",913485.0


In [9]:
housing_latest = zhvi_latest.merge(
    zori_latest,
    on=["RegionID", "RegionName", "RegionType", "StateName", "SizeRank", "date"],
    how="inner"
)

print("housing_latest shape:", housing_latest.shape)
housing_latest[["RegionName", "StateName", "home_value", "rent"]].head()


housing_latest shape: (546, 8)


,RegionName,StateName,home_value,rent
0,United States,NaN,361045.029486,2203.197513
1,"New York, NY",NY,721778.104533,3499.398403
2,"Los Angeles, CA",CA,990630.088957,4318.329317
3,"Chicago, IL",IL,354696.667711,2395.615001
4,"Dallas, TX",TX,362160.794805,2311.052245


In [10]:
df = housing_latest.merge(
    population,
    left_on="RegionName",
    right_on="metro_name",
    how="left"
)

print("Merged df shape:", df.shape)
print("Population missing:", df["population_2024"].isna().sum())
df.head()


Merged df shape: (546, 10)
Population missing: 546


,RegionID,SizeRank,RegionName,RegionType,StateName,date,home_value,rent,metro_name,population_2024
0,102001,0,United States,country,NaN,2025-11-30,361045.029486,2203.197513,NaN,NaN
1,394913,1,"New York, NY",msa,NY,2025-11-30,721778.104533,3499.398403,NaN,NaN
2,753899,2,"Los Angeles, CA",msa,CA,2025-11-30,990630.088957,4318.329317,NaN,NaN
3,394463,3,"Chicago, IL",msa,IL,2025-11-30,354696.667711,2395.615001,NaN,NaN
4,394514,4,"Dallas, TX",msa,TX,2025-11-30,362160.794805,2311.052245,NaN,NaN


In [11]:
df["monthly_net_income"] = MONTHLY_GROSS_INCOME * (1 - FEDERAL_EFFECTIVE_TAX - PAYROLL_TAX)

df["non_housing_monthly"] = NON_HOUSING_MONTHLY
df["savings_target"] = MONTHLY_SAVINGS_TARGET

df[["monthly_net_income", "non_housing_monthly", "savings_target"]].head()


,monthly_net_income,non_housing_monthly,savings_target
0,6195.833333,2340,833.333333
1,6195.833333,2340,833.333333
2,6195.833333,2340,833.333333
3,6195.833333,2340,833.333333
4,6195.833333,2340,833.333333


In [12]:
df["rent_burden"] = df["rent"] / MONTHLY_GROSS_INCOME

df["rent_leftover"] = (
    df["monthly_net_income"]
    - df["rent"]
    - df["non_housing_monthly"]
    - df["savings_target"]
)

df["rent_status"] = np.select(
    [
        df["rent_burden"] <= HOUSING_COMFORTABLE_RATIO,
        df["rent_burden"] <= HOUSING_STRETCHED_RATIO
    ],
    ["Comfortable", "Stretched"],
    default="Not Viable"
)

df["rent_status"].value_counts(dropna=False)


rent_status
Comfortable    469
Stretched       52
Not Viable      25
Name: count, dtype: int64

In [13]:
loan_amount = df["home_value"] * (1 - DOWN_PAYMENT_RATE)
monthly_rate = MORTGAGE_RATE / 12
n_payments = MORTGAGE_TERM_YEARS * 12

df["mortgage_payment_pi"] = (
    loan_amount
    * (monthly_rate * (1 + monthly_rate) ** n_payments)
    / ((1 + monthly_rate) ** n_payments - 1)
)

df["property_tax_monthly"] = df["home_value"] * PROPERTY_TAX_RATE / 12

df["mortgage_total_monthly"] = (
    df["mortgage_payment_pi"]
    + df["property_tax_monthly"]
    + HOME_INSURANCE_MONTHLY
)

df["mortgage_burden"] = df["mortgage_total_monthly"] / MONTHLY_GROSS_INCOME

df["mortgage_leftover"] = (
    df["monthly_net_income"]
    - df["mortgage_total_monthly"]
    - df["non_housing_monthly"]
    - df["savings_target"]
)

df["mortgage_status"] = np.select(
    [
        df["mortgage_burden"] <= HOUSING_COMFORTABLE_RATIO,
        df["mortgage_burden"] <= HOUSING_STRETCHED_RATIO
    ],
    ["Comfortable", "Stretched"],
    default="Not Viable"
)

df["mortgage_status"].value_counts(dropna=False)


mortgage_status
Comfortable    399
Stretched       85
Not Viable      62
Name: count, dtype: int64

In [14]:
zhvi_latest.columns


Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'date',
       'home_value'],
      dtype='object')